In [9]:
%load_ext dotenv
%dotenv

The dotenv extension is already loaded. To reload it, use:
  %reload_ext dotenv


In [11]:
# criar um dataset com dados de usuários do twitter + seus x tweets.

In [12]:
# packages imports here
import tweepy
import pandas as pd
from easydict import EasyDict as edict

In [13]:
# Loading variables for tweepy

twitter = {}
twitter['TWITTER_API_KEY'] = %env twitter_api_key 
twitter['TWITTER_API_SECRET'] = %env twitter_api_secret
twitter['TWITTER_API_TOKEN'] = %env twitter_access_token
twitter['TWITTER_API_TOKEN_SECRET'] = %env twitter_access_token_secret


#for key in twitter:
#    print(key, '->', twitter[key])

# Authenticate with Twitter API

twitter['TWITTER_API_TOKEN_SECRET']
auth = tweepy.OAuthHandler(twitter['TWITTER_API_KEY'], twitter['TWITTER_API_SECRET'])

# se não autenticar com o access token algumas funcionalidades da api 
# não ficam disponíveis. tipo verificar rate limits.
auth.set_access_token(twitter['TWITTER_API_TOKEN'], twitter['TWITTER_API_TOKEN_SECRET'])

# o objeto api é utilizado para realizar toda comunicação com a API do twitter.
api = tweepy.API(auth)

In [14]:
import botometer

rapidapi_key = %env rapidapi_key
twitter_app_auth = {
    'consumer_key': twitter['TWITTER_API_KEY'],
    'consumer_secret': twitter['TWITTER_API_SECRET'],
    'access_token': twitter['TWITTER_API_TOKEN'],
    'access_token_secret': twitter['TWITTER_API_TOKEN_SECRET']
  }

bom = botometer.Botometer(wait_on_ratelimit=True,
                          rapidapi_key=rapidapi_key,
                          **twitter_app_auth)

In [29]:
# Check a single account by screen name
r = edict(result = bom.check_account('@blckjzz'))

In [35]:
r.result

{'cap': {'english': 0.7384783166429812, 'universal': 0.5222398722127631},
 'display_scores': {'english': {'astroturf': 0.4,
   'fake_follower': 0.9,
   'financial': 0.2,
   'other': 1.7,
   'overall': 1.2,
   'self_declared': 0.0,
   'spammer': 0.2},
  'universal': {'astroturf': 0.4,
   'fake_follower': 0.4,
   'financial': 0.2,
   'other': 0.4,
   'overall': 0.6,
   'self_declared': 0.0,
   'spammer': 0.0}},
 'raw_scores': {'english': {'astroturf': 0.09,
   'fake_follower': 0.18,
   'financial': 0.04,
   'other': 0.34,
   'overall': 0.24,
   'self_declared': 0.0,
   'spammer': 0.04},
  'universal': {'astroturf': 0.08,
   'fake_follower': 0.07,
   'financial': 0.03,
   'other': 0.07,
   'overall': 0.11,
   'self_declared': 0.0,
   'spammer': 0.01}},
 'user': {'majority_lang': 'pt',
  'user_data': {'id_str': '1080370887357071362', 'screen_name': 'blckjzz'}}}

In [34]:
r.result.display_scores

{'english': {'astroturf': 0.4,
  'fake_follower': 0.9,
  'financial': 0.2,
  'other': 1.7,
  'overall': 1.2,
  'self_declared': 0.0,
  'spammer': 0.2},
 'universal': {'astroturf': 0.4,
  'fake_follower': 0.4,
  'financial': 0.2,
  'other': 0.4,
  'overall': 0.6,
  'self_declared': 0.0,
  'spammer': 0.0}}

In [ ]:

# Check a single account by id
result = bom.check_account(1548959833)


In [ ]:
# Check a sequence of accounts
accounts = ['@clayadavis', '@onurvarol', '@jabawack']
for screen_name, result in bom.check_accounts_in(accounts):
    # Do stuff with `screen_name` and `result`

In [8]:
# recuperando dados de um twitter user
def getDataFromUserByTwiiterHandle(handle):
    try:
        user = api.get_user(screen_name = handle)
        return user
    except tweepy.TweepError as e:
#         print("Tweepy Error retrieving user: {}".format(e))
        return e
#         return {'reason' : e.reason, 'codes' : e.api_code, 'response' : e.response, 'args' : e.args}

In [9]:
# recuperando dados de um twitter user
def getUserTimeline(uid, num_tweets = 100):
    try:       
        timeline = tweepy.Cursor(api.user_timeline, user_id = uid, count=num_tweets).items(num_tweets)
        return timeline
    except tweepy.TweepError as e:
#         print("Tweepy Error timeline: {}".format(e))
        return {'reason' : e.reason, 'codes' : e.api_code, 'response' : e.response, 'args' : e.args}

In [10]:
# create user dataframe by list of users retrieved by tweepy
def createDataFrameFromUser(user, dfUsers, error=False, handle=None):
#     dfUsers = pd.DataFrame();
#     for user in users:
    if error == False:
        u = {
            'twitter_id': user.id,
            'name': user.name,
            'handle' : user.screen_name,
            'twitter_is_protected': user.protected,
            'description': user.description,
            'followers_count': user.followers_count,
            'friends_count': user.friends_count,
            'location': user.location,
            'created_at': user.created_at,
            'verified': user.verified,
            'lang': user.lang,
            'default_profile': user.default_profile,
            'profile_image': user.profile_image_url,
            'withheld_in_countries' : user.withheld_in_countries
        }
        dfUsers = dfUsers.append(u, ignore_index=True)
    else:
        e = {'error': user, 'handle' : handle}
        dfUsers = dfUsers.append(e, ignore_index=True)
    return dfUsers

In [12]:
def createDataFrameFromTimeline(timeline, dfTimeline, error = False, handle=None):
    if error == False:
        for tweet in timeline:
            t = {
                'tweet_id' : tweet.id,
                'tweet_author': tweet.author.screen_name,
                'tweet_id_str' : tweet.id_str,
                'tweet_retweeted' : tweet.retweeted,
                'tweet_created_at': tweet.created_at,
                'tweet_source' : tweet.source,
                'tweet_author_id_str':tweet.author.id_str,
                'tweet_text':tweet.text,
                'tweet_contributors': tweet.contributors,
                'tweet_favorite_count':tweet.favorite_count,
                'tweet_favorited' : tweet.favorited,
                'tweet_geo' : tweet.geo,
                'tweet_is_retweet': tweet.is_quote_status,
                'tweet_lang' : tweet.lang,
                'tweet_place' : tweet.place,
                'tweet_hashtags' : list(map(lambda x:x['text'], tweet.entities['hashtags']))
            }
            dfTimeline = dfTimeline.append(t, ignore_index=True)
    else:
        e = {'error': timeline, 'handle' : handle}
        dfTimeline = dfTimeline.append(e, ignore_index=True)
    return dfTimeline    

In [13]:
twitterUser = getDataFromUserByTwiiterHandle("blckjzzzzz")
print(type(twitterUser))

<class 'tweepy.error.TweepError'>


In [116]:
#handles = pd.read_csv("/Users/dc/Downloads/teste.csv", skiprows=[0], header=None,
df2 = pd.read_csv("/Users/dc/Downloads/inctdd-amostrabots2.csv",
                      delimiter=",", encoding="utf-8")

df2.head()

# len(df2)

,Unnamed: 0,tabelaAmostra,p,É Bot?,"Se você fosse atribuir uma função ao bot, qual seria?",Função #2,Comportamento agressivo?,Comportamento repetitivo com # ou menções?,Parece só Retweetar?,Só compartilha links?,Só faz comentários?,Enaltece muito outros usuários?,Faz muito uso de emojis?,Tem muitos posts sem textos?,Unnamed: 14
0,1,https://twitter.com/@lemathes,0000.csv,não,não se aplica,NaN,não,não,não,não,não,não,não,não,NaN
1,2,https://twitter.com/@Maurcio98905595,0000.csv,não,não se aplica,NaN,não,não,não,não,não,não,não,não,NaN
2,3,https://twitter.com/@LunViana,0000.csv,não,não se aplica,NaN,não,não,não,não,não,não,não,não,NaN
3,4,https://twitter.com/@felipeleixas,0000.csv,sim,Publicar hashtags,Atacar,sim,sim,não,não,não,não,não,não,NaN
4,5,https://twitter.com/@JoseCar41451194,0000.csv,Não,não se aplica,NaN,não,não,não,não,não,não,não,não,NaN


In [145]:
df2[['delete','handle']] = df2.tabelaAmostra.str.split("@",expand=True,)
df2 = df2.drop(columns='delete')
df2.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1074 entries, 0 to 1073
Data columns (total 16 columns):
 #   Column                                                 Non-Null Count  Dtype 
---  ------                                                 --------------  ----- 
 0   Unnamed: 0                                             1074 non-null   int64 
 1   tabelaAmostra                                          1074 non-null   object
 2   p                                                      1074 non-null   object
 3   É Bot?                                                 1074 non-null   object
 4   Se você fosse atribuir uma função ao bot, qual seria?  1074 non-null   object
 5   Função #2                                              189 non-null    object
 6   Comportamento agressivo?                               1074 non-null   object
 7   Comportamento repetitivo com # ou menções?             1074 non-null   object
 8   Parece só Retweetar?                                   107

In [151]:
dfsplit = pd.DataFrame()
dfsplit[['delete','handle']] = df2.tabelaAmostra.str.split("@",expand=True,)
dfsplit = dfsplit.drop(columns='delete')
dfsplit.head()

,handle
0,lemathes
1,Maurcio98905595
2,LunViana
3,felipeleixas
4,JoseCar41451194


In [158]:
# dataframes creation

columns = {
    'error'
}

dfUser = pd.DataFrame(columns=columns);
dfTimelines = pd.DataFrame(columns=columns);

#for handle in handles:
# handles = {'blckjxxxxxxzz', 'VieiraClever1', 'blckjzz'} # testing handles {do not exist, blocked timeline, real}

print(f' ############ {len(dfsplit)} # PERFIS PARA ANALISAR')
for index, handle in dfsplit.itertuples(): 
#     print(f'{index+1}    #    Analisando o perfil:@ {handle}')
    twitterUser = getDataFromUserByTwiiterHandle(handle)
    # if its true, the twitter api has returned some error
    # False, should mean that the user is found and you can work with it.
#     print(type(twitterUser))
    if isinstance(twitterUser, tweepy.error.TweepError) == False: # means that the user exists because the Twitter API response
        print(f'Analisando o @{handle}. Nome: {twitterUser.name}')
        # When you have the user retrieved
        # The Expected flow should be written here
        dfUser = createDataFrameFromUser(twitterUser, dfUser)
        if twitterUser.protected == False: #means that the timeline is open and you should be allowed to retrieve data
            response = getUserTimeline(twitterUser.id_str)
            # all analysed user will be stored to the same dataframe
            # and its tweets will be agragated into a dataframe within all users tweets retrived.
            dfTimelines = createDataFrameFromTimeline(response, dfTimelines) # should add data to the main dataframe
#         else: # timeline closed just log the error and move on
# #             response = getUserTimeline(twitterUser.id_str)
# #             print(f'{count} Timeline de :@ {handle} é fechada!')
#             dfTimelines = createDataFrameFromTimeline("Timeline bloqueada", dfTimelines, error=True, handle=handle) # should add data to the main dataframe
#             # throw error.
#             # log error in the table
#             # move to the next line
# #             break    
    else:
#         print(f'{count} O Usuário @{handle} não rolou. Razão: {twitterUser.reason}')
        dfUser = createDataFrameFromUser(twitterUser.reason, dfUser, error=True, handle = handle)
        # move to the next line of the handle dataset
        # log some error into the dataset
        # continue the flux

 ############ 1074 # PERFIS PARA ANALISAR
1    #    Analisando o perfil:@ lemathes
2    #    Analisando o perfil:@ Maurcio98905595
3    #    Analisando o perfil:@ LunViana
4    #    Analisando o perfil:@ felipeleixas
5    #    Analisando o perfil:@ JoseCar41451194
6    #    Analisando o perfil:@ stefmilhori
7    #    Analisando o perfil:@ Maurio0916
8    #    Analisando o perfil:@ alaincremonezi
9    #    Analisando o perfil:@ marctrickguedes
10    #    Analisando o perfil:@ Valdir_25
11    #    Analisando o perfil:@ HermesMachadoAP
12    #    Analisando o perfil:@ euclaudemir
13    #    Analisando o perfil:@ LongoMarlongo
14    #    Analisando o perfil:@ Tadeu88537223
15    #    Analisando o perfil:@ EliasBispodeCe1
16    #    Analisando o perfil:@ lessa_tadeu
17    #    Analisando o perfil:@ kentyan71
18    #    Analisando o perfil:@ Sirenite1
19    #    Analisando o perfil:@ elechimamil
20    #    Analisando o perfil:@ mhelena17
21    #    Analisando o perfil:@ victorcalazans
22    

178    #    Analisando o perfil:@ waldema93547310
179    #    Analisando o perfil:@ edsonbjl
180    #    Analisando o perfil:@ RomagnaniLite
181    #    Analisando o perfil:@ DeniseLuz3
182    #    Analisando o perfil:@ VagnerMouraCam1
183    #    Analisando o perfil:@ leandrofmax
184    #    Analisando o perfil:@ jjggukie__
185    #    Analisando o perfil:@ kalaumusico
186    #    Analisando o perfil:@ g_della_
187    #    Analisando o perfil:@ DILMAFREITAS333
188    #    Analisando o perfil:@ AragaoJosafa
189    #    Analisando o perfil:@ Tony18822268
190    #    Analisando o perfil:@ MartoniTony
191    #    Analisando o perfil:@ TniaMar90422263
192    #    Analisando o perfil:@ crbastelli
193    #    Analisando o perfil:@ priscil16465789
194    #    Analisando o perfil:@ PAULAAL37165233
195    #    Analisando o perfil:@ Ronerio9
196    #    Analisando o perfil:@ Paulokastro70
197    #    Analisando o perfil:@ ana99157293
198    #    Analisando o perfil:@ Marciocarlosro1
199    #    

353    #    Analisando o perfil:@ RenatoCoelhoFre
354    #    Analisando o perfil:@ Juliano50617961
355    #    Analisando o perfil:@ AndrCam70781993
356    #    Analisando o perfil:@ pedro_paravito
357    #    Analisando o perfil:@ cyberhannya
358    #    Analisando o perfil:@ FavinhaMarcela
359    #    Analisando o perfil:@ Jocadeol
360    #    Analisando o perfil:@ AControld
361    #    Analisando o perfil:@ IverTinny
362    #    Analisando o perfil:@ DarlanSattler
363    #    Analisando o perfil:@ limallanitta
364    #    Analisando o perfil:@ lombardijc
365    #    Analisando o perfil:@ LuizCarlosdeAb7
366    #    Analisando o perfil:@ MrciaSueliLope1
367    #    Analisando o perfil:@ SoaresOg
368    #    Analisando o perfil:@ DanielOgelio
369    #    Analisando o perfil:@ Anubis66150815
370    #    Analisando o perfil:@ Liliane73428366
371    #    Analisando o perfil:@ Eduardosobrosa
372    #    Analisando o perfil:@ BarbosaVoss
373    #    Analisando o perfil:@ pjiart
374    #  

527    #    Analisando o perfil:@ larimont00
528    #    Analisando o perfil:@ ZSantossaz
529    #    Analisando o perfil:@ bosswooy
530    #    Analisando o perfil:@ BolsonaroMessi4
531    #    Analisando o perfil:@ daysecoficial
532    #    Analisando o perfil:@ kthdetaills
533    #    Analisando o perfil:@ Marcelo84048142
534    #    Analisando o perfil:@ guilher81248250
535    #    Analisando o perfil:@ Paulolo98015633
536    #    Analisando o perfil:@ PauloMBrasil
537    #    Analisando o perfil:@ jsgirl
538    #    Analisando o perfil:@ AvilaAlcindo
539    #    Analisando o perfil:@ bruno64855465
540    #    Analisando o perfil:@ BielFenix7
541    #    Analisando o perfil:@ 3nzW0ooo
542    #    Analisando o perfil:@ deizetodoroki
543    #    Analisando o perfil:@ Alvonaro
544    #    Analisando o perfil:@ thome_edna
545    #    Analisando o perfil:@ Dr_Jr_Sarilho
546    #    Analisando o perfil:@ PaulinhaLandimd
547    #    Analisando o perfil:@ joseantoniomas7
548    #    Analis

701    #    Analisando o perfil:@ _rosy_38
702    #    Analisando o perfil:@ JORNALBRASILDE1
703    #    Analisando o perfil:@ ssa_mariana
704    #    Analisando o perfil:@ luciano280611
705    #    Analisando o perfil:@ BazilioAlex
706    #    Analisando o perfil:@ Ruineves27
707    #    Analisando o perfil:@ dixxyugyeom
708    #    Analisando o perfil:@ Prestonejr
709    #    Analisando o perfil:@ bysoocut
710    #    Analisando o perfil:@ PG21A
711    #    Analisando o perfil:@ sabricream
712    #    Analisando o perfil:@ favboyjinnie
713    #    Analisando o perfil:@ Everton__080820
714    #    Analisando o perfil:@ comentobbb2021
715    #    Analisando o perfil:@ LauraSi66030021
716    #    Analisando o perfil:@ Severin11496561
717    #    Analisando o perfil:@ semfenoBrasil
718    #    Analisando o perfil:@ Cleiton58410322
719    #    Analisando o perfil:@ Suiane12406352
720    #    Analisando o perfil:@ Solange40232969
721    #    Analisando o perfil:@ 3li4stt
722    #    Analis

873    #    Analisando o perfil:@ CarlosA76979782
874    #    Analisando o perfil:@ CabralMakiyoshi
875    #    Analisando o perfil:@ Guilhe22821397
876    #    Analisando o perfil:@ PauloCzardaSil2
877    #    Analisando o perfil:@ AlexandreRicht7
878    #    Analisando o perfil:@ ricardomelo1968
879    #    Analisando o perfil:@ DilaSoares9
880    #    Analisando o perfil:@ NeusaMarquess
881    #    Analisando o perfil:@ ReginaConte3
882    #    Analisando o perfil:@ RenataMartinsR3
883    #    Analisando o perfil:@ Antonio24529020
884    #    Analisando o perfil:@ NATHANI18729150
885    #    Analisando o perfil:@ RachedJorge
886    #    Analisando o perfil:@ lovelyfritz_
887    #    Analisando o perfil:@ sparkyviana
888    #    Analisando o perfil:@ iEatSanie
889    #    Analisando o perfil:@ ThePattryota
890    #    Analisando o perfil:@ luanlino__
891    #    Analisando o perfil:@ Guile_Phoenix38
892    #    Analisando o perfil:@ Alexand85602241
893    #    Analisando o perfil:@ D

1047    #    Analisando o perfil:@ LuizAugustoPai4
1048    #    Analisando o perfil:@ chocopoemlate16
1049    #    Analisando o perfil:@ Joonbabykoya
1050    #    Analisando o perfil:@ zoldyevvil
1051    #    Analisando o perfil:@ predadoalfa
1052    #    Analisando o perfil:@ FePatriota1
1053    #    Analisando o perfil:@ NandaAndretto
1054    #    Analisando o perfil:@ safetyjm
1055    #    Analisando o perfil:@ CarlosG82785363
1056    #    Analisando o perfil:@ KP62A
1057    #    Analisando o perfil:@ marstwolf
1058    #    Analisando o perfil:@ Marcos_11_66
1059    #    Analisando o perfil:@ Rosiveti1
1060    #    Analisando o perfil:@ uzusaske
1061    #    Analisando o perfil:@ vhsmessy
1062    #    Analisando o perfil:@ JMBBrasil
1063    #    Analisando o perfil:@ baia_canuto
1064    #    Analisando o perfil:@ pjmackerman
1065    #    Analisando o perfil:@ EN30A
1066    #    Analisando o perfil:@ clara_kess
1067    #    Analisando o perfil:@ CesarNi85939384
1068    #    Analisand

In [159]:
dfUser.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1074 entries, 0 to 1073
Data columns (total 15 columns):
 #   Column                 Non-Null Count  Dtype         
---  ------                 --------------  -----         
 0   error                  173 non-null    object        
 1   created_at             901 non-null    datetime64[ns]
 2   default_profile        901 non-null    object        
 3   description            901 non-null    object        
 4   followers_count        901 non-null    float64       
 5   friends_count          901 non-null    float64       
 6   handle                 1074 non-null   object        
 7   lang                   0 non-null      object        
 8   location               901 non-null    object        
 9   name                   901 non-null    object        
 10  profile_image          901 non-null    object        
 11  twitter_id             901 non-null    float64       
 12  twitter_is_protected   901 non-null    object        
 13  ver

In [160]:
dfTimelines.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 87527 entries, 0 to 87526
Data columns (total 18 columns):
 #   Column                Non-Null Count  Dtype         
---  ------                --------------  -----         
 0   error                 13 non-null     object        
 1   tweet_author          87514 non-null  object        
 2   tweet_author_id_str   87514 non-null  object        
 3   tweet_contributors    0 non-null      object        
 4   tweet_created_at      87514 non-null  datetime64[ns]
 5   tweet_favorite_count  87514 non-null  float64       
 6   tweet_favorited       87514 non-null  object        
 7   tweet_geo             20 non-null     object        
 8   tweet_hashtags        87514 non-null  object        
 9   tweet_id              87514 non-null  float64       
 10  tweet_id_str          87514 non-null  object        
 11  tweet_is_retweet      87514 non-null  object        
 12  tweet_lang            87514 non-null  object        
 13  tweet_place     

In [163]:
df2.to_csv("/Users/dc/Downloads/handles_inct.csv", sep=',', encoding="utf-8")

In [ ]:
dfUser.to_csv("/Users/dc/Downloads/df_users_inct.csv", sep=',', encoding="utf-8")

In [162]:
dfTimelines.to_csv("/Users/dc/Downloads/df_timelines_inct.csv", sep=',', encoding="utf-8")

In [570]:
dfTimelines.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 130 entries, 0 to 129
Data columns (total 18 columns):
 #   Column                Non-Null Count  Dtype         
---  ------                --------------  -----         
 0   error                 14 non-null     object        
 1   handle                14 non-null     object        
 2   tweet_author          116 non-null    object        
 3   tweet_author_id_str   116 non-null    object        
 4   tweet_contributors    0 non-null      object        
 5   tweet_created_at      116 non-null    datetime64[ns]
 6   tweet_favorite_count  116 non-null    float64       
 7   tweet_favorited       116 non-null    object        
 8   tweet_geo             0 non-null      object        
 9   tweet_hashtags        116 non-null    object        
 10  tweet_id              116 non-null    float64       
 11  tweet_id_str          116 non-null    object        
 12  tweet_is_retweet      116 non-null    object        
 13  tweet_lang          